In [0]:
# imports 
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline 
from pyspark.ml.regression import RandomForestRegressor 
from pyspark.ml.evaluation import RegressionEvaluator
import mlflow 
from mlflow.tracking import MlflowClient
import pandas as pd
from pyspark.ml.feature import RFormula

client = MlflowClient()

# load the datasets 
userhome = 'dbfs:/user/skim658@gwu.edu'

redDF = spark.read.parquet(userhome + '/final-project/reddf.parquet')
whiteDF = spark.read.parquet(userhome + '/final-project/whitedf.parquet')

In [0]:
# split the dataset into train and test sets 
redTrainDF, redTestDF = redDF.repartition(8).randomSplit([0.8, 0.2], seed = 42)
print(redTrainDF.cache().count())
whiteTrainDF, whiteTestDF = whiteDF.repartition(8).randomSplit([0.8, 0.2], seed = 42)
print(whiteTrainDF.cache().count())

1276
3919

## Random Forest with Cross Validation 

Max Depth: 2, 5, 10; Num Trees: 10, 20, 100

## Red Wine

In [0]:
# log random forest regressor model 
with mlflow.start_run(run_name = 'RED-RF-All-Features') as run:
  # model 
  redRF = RandomForestRegressor(labelCol = 'quality')
  vecAssemblerRF = VectorAssembler(inputCols = [x for x in redTrainDF.columns if x != 'quality'], outputCol = 'features')
  # cross validation 
  rfParamGrid = ParamGridBuilder().addGrid(redRF.maxDepth, [2, 5, 10]).addGrid(redRF.numTrees, [10, 20, 100]).build()
  evaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
  cv = CrossValidator(estimator = redRF, evaluator = evaluator, estimatorParamMaps = rfParamGrid, numFolds = 3, parallelism = 4)
  
  # pipeline 
  pipeline = Pipeline(stages = [vecAssemblerRF, cv])
  pipelineModel = pipeline.fit(redTrainDF)
  
  cvModel = pipelineModel.stages[-1]
  rfModel = cvModel.bestModel 
  
  # create and evaluate predictions 
  redPredDF = pipelineModel.transform(redTestDF)
  display(redPredDF.select('features', 'quality', 'prediction'))
  
  # log parameters
  mlflow.log_param('features', 'all')
  mlflow.log_param('color', 'red')
  # log metric
  rmse = evaluator.setMetricName('rmse').evaluate(redPredDF)
  mlflow.log_metric('rmse', rmse)
  

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

features,quality,prediction
"Map(vectorType -> dense, length -> 11, values -> List(5.4, 0.74, 0.0, 1.2, 0.041, 16.0, 46.0, 0.99258, 4.01, 0.59, 12.5))",6,5.93423442814284
"Map(vectorType -> dense, length -> 11, values -> List(5.9, 0.61, 0.08, 2.1, 0.071, 16.0, 24.0, 0.99376, 3.56, 0.77, 11.1))",6,5.946039611242907
"Map(vectorType -> dense, length -> 11, values -> List(6.0, 0.5, 0.0, 1.4, 0.057, 15.0, 26.0, 0.99448, 3.36, 0.45, 9.5))",5,5.042511296387279
"Map(vectorType -> dense, length -> 11, values -> List(6.2, 0.64, 0.09, 2.5, 0.081, 15.0, 26.0, 0.99538, 3.57, 0.63, 12.0))",5,6.126822591147557
"Map(vectorType -> dense, length -> 11, values -> List(6.4, 0.56, 0.15, 1.8, 0.078, 17.0, 65.0, 0.99294, 3.33, 0.6, 10.5))",6,5.713720241453626
"Map(vectorType -> dense, length -> 11, values -> List(6.6, 0.815, 0.02, 2.7, 0.072, 17.0, 34.0, 0.9955, 3.58, 0.89, 12.3))",7,6.394349623698464
"Map(vectorType -> dense, length -> 11, values -> List(6.8, 0.91, 0.06, 2.0, 0.06, 4.0, 11.0, 0.99592, 3.53, 0.64, 10.9))",4,5.552409827055385
"Map(vectorType -> dense, length -> 11, values -> List(7.0, 0.36, 0.21, 2.4, 0.086, 24.0, 69.0, 0.99556, 3.4, 0.53, 10.1))",6,5.464368203044319
"Map(vectorType -> dense, length -> 11, values -> List(7.0, 0.685, 0.0, 1.9, 0.067, 40.0, 63.0, 0.9979, 3.6, 0.81, 9.9))",5,5.28340437170132
"Map(vectorType -> dense, length -> 11, values -> List(7.0, 0.78, 0.08, 2.0, 0.093, 10.0, 19.0, 0.9956, 3.4, 0.47, 10.0))",5,5.019069493173255


In [0]:
# see the best model parameters 
paramDict = rfModel.extractParamMap()
tempDict = {}
for k, v in paramDict.items():
  tempDict[k.name] = v

bestMaxDepth = tempDict['maxDepth']
bestnumTrees = tempDict['numTrees']

tempDict = {'Max Depth': [bestMaxDepth], 'Num Trees': [bestnumTrees]}
bestCVModel = pd.DataFrame(tempDict, index = [0])
display(bestCVModel)

Max Depth,Num Trees
10,100


In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['mlModelClass'] == 'RandomForestRegressor' and int(run.data.params['numTrees']) == bestnumTrees and int(run.data.params['maxDepth']) == bestMaxDepth:
    print(run) 
    redMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(redMetricsDF)

<Run: data=<RunData: metrics={'avg_rmse': 0.6099007342616175, 'std_rmse': 0.016669344521339308}, params={'maxDepth': '10',
 'mlEstimatorUid': 'RandomForestRegressor_7663956b120c',
 'mlModelClass': 'RandomForestRegressor',
 'numTrees': '100'}, tags={'fit_uuid': '958615',
 'mlflow.parentRunId': '4411f3a6eda14d4794097aeb7b01ffc0',
 'mlflow.rootRunId': '4411f3a6eda14d4794097aeb7b01ffc0',
 'mlflow.user': 'prodbyskimmer@gmail.com',
 'runSource': 'mllibAutoTracking'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302293044/3e146c392cc0499b92c4813243a44f41/artifacts', end_time=None, experiment_id='929351302293044', lifecycle_stage='active', run_id='3e146c392cc0499b92c4813243a44f41', run_uuid='3e146c392cc0499b92c4813243a44f41', start_time=1606255013479, status='RUNNING', user_id=''>>

avg_rmse,std_rmse
0.6099007342616175,0.016669344521339308


In [0]:
# feature importance 
pandasDF = pd.DataFrame(list(zip([x for x in redTrainDF.columns if x != 'quality'], rfModel.featureImportances)), columns=["feature", "importance"])
topFeatures = pandasDF.sort_values(["importance"], ascending=False)
topFeatures

Out[6]:

,feature,importance
10,alcohol,0.235373
9,sulphates,0.165005
1,volatile_acidity,0.120866
6,total_sulfur_dioxide,0.078331
0,fixed_acidity,0.068922
4,chlorides,0.064406
7,density,0.064012
8,pH,0.053194
5,free_sulfur_dioxide,0.051725
2,citric_acid,0.050607


## White Wine

In [0]:
# log random forest regressor model 
with mlflow.start_run(run_name = 'WHITE-RF-All-Features') as run:
  # model 
  whiteRF = RandomForestRegressor(labelCol = 'quality')
  vecAssemblerRF = VectorAssembler(inputCols = [x for x in whiteTrainDF.columns if x != 'quality'], outputCol = 'features')
  # cross validation 
  rfParamGrid = ParamGridBuilder().addGrid(whiteRF.maxDepth, [2, 5, 10]).addGrid(whiteRF.numTrees, [10, 20, 100]).build()
  evaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
  cv = CrossValidator(estimator = whiteRF, evaluator = evaluator, estimatorParamMaps = rfParamGrid, numFolds = 3, parallelism = 4)
  
  # pipeline 
  pipeline = Pipeline(stages = [vecAssemblerRF, cv])
  pipelineModel = pipeline.fit(whiteTrainDF)
  
  cvModel = pipelineModel.stages[-1]
  rfModel = cvModel.bestModel 
  
  # create and evaluate predictions 
  whitePredDF = pipelineModel.transform(whiteTestDF)
  display(whitePredDF.select('features', 'quality', 'prediction'))
  
  # log parameters
  mlflow.log_param('features', 'all')
  mlflow.log_param('color', 'white')
  # log metric
  rmse = evaluator.setMetricName('rmse').evaluate(whitePredDF)
  mlflow.log_metric('rmse', rmse)
  

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

features,quality,prediction
"Map(vectorType -> dense, length -> 11, values -> List(4.9, 0.47, 0.17, 1.9, 0.035, 60.0, 148.0, 0.98964, 3.27, 0.35, 11.5))",6,6.378933526347721
"Map(vectorType -> dense, length -> 11, values -> List(5.1, 0.35, 0.26, 6.8, 0.034, 36.0, 120.0, 0.99188, 3.38, 0.4, 11.5))",6,6.370790906744824
"Map(vectorType -> dense, length -> 11, values -> List(5.2, 0.3, 0.34, 1.5, 0.038, 18.0, 96.0, 0.98942, 3.56, 0.48, 13.0))",8,6.614513653705502
"Map(vectorType -> dense, length -> 11, values -> List(5.4, 0.29, 0.38, 1.2, 0.029, 31.0, 132.0, 0.98895, 3.28, 0.36, 12.4))",6,6.368252371060446
"Map(vectorType -> dense, length -> 11, values -> List(5.6, 0.18, 0.27, 1.7, 0.03, 31.0, 103.0, 0.98892, 3.35, 0.37, 12.9))",6,6.726008184447489
"Map(vectorType -> dense, length -> 11, values -> List(5.6, 0.26, 0.18, 1.4, 0.034, 18.0, 135.0, 0.99174, 3.32, 0.35, 10.2))",6,5.6574508151327905
"Map(vectorType -> dense, length -> 11, values -> List(5.7, 0.2, 0.3, 2.5, 0.046, 38.0, 125.0, 0.99276, 3.34, 0.5, 9.9))",6,5.938171655128006
"Map(vectorType -> dense, length -> 11, values -> List(5.7, 0.28, 0.35, 1.2, 0.052, 39.0, 141.0, 0.99108, 3.44, 0.69, 11.3))",6,6.37545221691866
"Map(vectorType -> dense, length -> 11, values -> List(5.8, 0.22, 0.29, 1.3, 0.036, 25.0, 68.0, 0.98865, 3.24, 0.35, 12.6))",6,6.302192592568122
"Map(vectorType -> dense, length -> 11, values -> List(5.8, 0.23, 0.27, 1.8, 0.043, 24.0, 69.0, 0.9933, 3.38, 0.31, 9.4))",6,5.788330841691096


In [0]:
# see the best model parameters 
paramDict = rfModel.extractParamMap()
tempDict = {}
for k, v in paramDict.items():
  tempDict[k.name] = v

bestMaxDepth = tempDict['maxDepth']
bestnumTrees = tempDict['numTrees']

tempDict = {'Max Depth': [bestMaxDepth], 'Num Trees': [bestnumTrees]}
bestCVModel = pd.DataFrame(tempDict, index = [0])
display(bestCVModel)

Max Depth,Num Trees
10,100


In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['mlModelClass'] == 'RandomForestRegressor' and int(run.data.params['numTrees']) == bestnumTrees and int(run.data.params['maxDepth']) == bestMaxDepth:
    print(run) 
    whiteMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(whiteMetricsDF)

<Run: data=<RunData: metrics={'avg_rmse': 0.6535970142964916, 'std_rmse': 0.01902234949296995}, params={'maxDepth': '10',
 'mlEstimatorUid': 'RandomForestRegressor_a0efaabbdc44',
 'mlModelClass': 'RandomForestRegressor',
 'numTrees': '100'}, tags={'fit_uuid': 'a31857',
 'mlflow.parentRunId': '6d7b80074a2b442fb33d3130aa49a025',
 'mlflow.rootRunId': '6d7b80074a2b442fb33d3130aa49a025',
 'mlflow.user': 'prodbyskimmer@gmail.com',
 'runSource': 'mllibAutoTracking'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302293044/8a528bca42b84cf086b0cea571a129bd/artifacts', end_time=None, experiment_id='929351302293044', lifecycle_stage='active', run_id='8a528bca42b84cf086b0cea571a129bd', run_uuid='8a528bca42b84cf086b0cea571a129bd', start_time=1606255447746, status='RUNNING', user_id=''>>

avg_rmse,std_rmse
0.6535970142964916,0.01902234949296995


In [0]:
# feature importance 
pandasDF = pd.DataFrame(list(zip([x for x in redTrainDF.columns if x != 'quality'], rfModel.featureImportances)), columns=["feature", "importance"])
topFeatures = pandasDF.sort_values(["importance"], ascending=False)
topFeatures

Out[10]:

,feature,importance
10,alcohol,0.221868
1,volatile_acidity,0.121461
5,free_sulfur_dioxide,0.108984
7,density,0.100680
4,chlorides,0.081319
8,pH,0.065586
6,total_sulfur_dioxide,0.064543
2,citric_acid,0.061542
0,fixed_acidity,0.060646
3,residual_sugar,0.060332


## Random Forests with Hyperopt Hyperparameter Tuning

In [0]:
# imports 
from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, Trials 

/databricks/python/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
 from imp import reload

In [0]:
# define the search space 
search_space = {
  'maxDepth': hp.randint('maxDepth', 2, 30),
  'maxBins': hp.randint('maxBins', 10, 40),
  'numTrees': hp.randint('numTrees', 10, 100)
}

## Red Wine

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    # hyperparameters to tune 
    maxDepth = params['maxDepth']
    maxBins = params['maxBins']
    numTrees = params['numTrees']
    
    # model 
    vecAssemblerRF = VectorAssembler(inputCols = [x for x in redTrainDF.columns if x != 'quality'], outputCol = 'features')
    rf = RandomForestRegressor(labelCol = 'quality', maxBins = maxBins, maxDepth = maxDepth, numTrees = numTrees, seed = 42)
    
    # pipeline 
    pipeline = Pipeline(stages = [vecAssemblerRF, rf])
    pipelineModel = pipeline.fit(redTrainDF)
    
    # evaluate predictions 
    redPredDF = pipelineModel.transform(redTestDF)
    regressionEvaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
    rmse = regressionEvaluator.evaluate(redPredDF)
    
    # log parameters
    mlflow.log_param('maxDepth', maxDepth)
    mlflow.log_param('maxBins', maxBins)
    mlflow.log_param('numTrees', numTrees)
    mlflow.log_param('color', 'red')
    mlflow.log_param('features', 'all')
    mlflow.log_param('tuning', 'hyperopt')
    mlflow.log_metric('rmse', rmse)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'RED-RF-All-Features-Hyperopt') as run:
  num_evals = 30
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = search_space,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  # log param and metric for best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])

0%| | 0/30 [00:00<?, ?trial/s, best loss=?] 3%|▎ | 1/30 [00:39<19:12, 39.73s/trial, best loss: 0.6185897326311957] 7%|▋ | 2/30 [02:28<28:09, 60.35s/trial, best loss: 0.5916288861285772] 10%|█ | 3/30 [03:09<24:32, 54.55s/trial, best loss: 0.5916288861285772] 13%|█▎ | 4/30 [03:39<20:27, 47.23s/trial, best loss: 0.5916288861285772] 17%|█▋ | 5/30 [07:16<40:52, 98.09s/trial, best loss: 0.5916288861285772] 20%|██ | 6/30 [10:34<51:14, 128.09s/trial, best loss: 0.5916288861285772] 23%|██▎ | 7/30 [12:55<50:34, 131.95s/trial, best loss: 0.5883215924223659] 27%|██▋ | 8/30 [16:43<59:01, 160.98s/trial, best loss: 0.5883215924223659] 30%|███ | 9/30 [18:55<53:14, 152.11s/trial, best loss: 0.5883215924223659] 33%|███▎ | 10/30 [21:32<51:12, 153.61s/trial, best loss: 0.5883215924223659] 37%|███▋ | 11/30 [22:47<41:08, 129.94s/trial, best loss: 0.5883215924223659] 40%|████ | 12/30 [23:51<33:05, 110.28s/trial, best loss: 0.5883215924223659] 43%|████▎ | 13/30 [23:57<22:23, 79.01s/trial, best loss: 0.5883215924223659]  47%|████▋ | 14/30 [25:59<24:31, 91.99s/trial, best loss: 0.5883215924223659] 50%|█████ | 15/30 [26:06<16:34, 66.33s/trial, best loss: 0.5883215924223659] 53%|█████▎ | 16/30 [26:18<11:40, 50.04s/trial, best loss: 0.5883215924223659] 57%|█████▋ | 17/30 [26:24<08:00, 36.98s/trial, best loss: 0.5883215924223659] 60%|██████ | 18/30 [26:32<05:37, 28.11s/trial, best loss: 0.5883215924223659] 63%|██████▎ | 19/30 [30:40<17:16, 94.19s/trial, best loss: 0.5883215924223659] 67%|██████▋ | 20/30 [31:00<11:58, 71.84s/trial, best loss: 0.5883215924223659] 70%|███████ | 21/30 [33:19<13:47, 91.91s/trial, best loss: 0.5883215924223659] 73%|███████▎ | 22/30 [33:30<09:02, 67.83s/trial, best loss: 0.5883215924223659] 77%|███████▋ | 23/30 [34:11<06:57, 59.70s/trial, best loss: 0.5883215924223659] 80%|████████ | 24/30 [34:29<04:42, 47.09s/trial, best loss: 0.5883215924223659] 83%|████████▎ | 25/30 [34:49<03:15, 39.13s/trial, best loss: 0.5883215924223659] 87%|████████▋ | 26/30 [37:07<04:34, 68.61s/trial, best loss: 0.5883215924223659] 90%|█████████ | 27/30 [39:41<04:43, 94.42s/trial, best loss: 0.5858820704844101] 93%|█████████▎| 28/30 [41:17<03:09, 94.76s/trial, best loss: 0.5858820704844101] 97%|█████████▋| 29/30 [43:25<01:44, 104.67s/trial, best loss: 0.5858820704844101]100%|██████████| 30/30 [43:31<00:00, 75.07s/trial, best loss: 0.5858820704844101] 100%|██████████| 30/30 [43:31<00:00, 87.03s/trial, best loss: 0.5858820704844101]

In [0]:
redResult = pd.DataFrame(best_hyperparam, index = [0])
display(redResult)

maxBins,maxDepth,numTrees
14,26,54


In [0]:
bestMaxBins = best_hyperparam['maxBins']
bestMaxDepth = best_hyperparam['maxDepth']
bestNumTrees = best_hyperparam['numTrees']

In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['tuning'] == 'hyperopt' and int(run.data.params['maxBins']) == bestMaxBins and int(run.data.params['maxDepth']) == bestMaxDepth and int(run.data.params['numTrees']) == bestNumTrees:
    print(run) 
    redMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(redMetricsDF)

<Run: data=<RunData: metrics={'rmse': 0.5858820704844101}, params={'color': 'red',
 'features': 'all',
 'maxBins': '14',
 'maxDepth': '26',
 'numTrees': '54',
 'tuning': 'hyperopt'}, tags={'mlflow.databricks.cluster.id': '1124-212020-lark668',
 'mlflow.databricks.cluster.info': '{"cluster_id":"1124-212020-lark668","creator_user_id":2440320843892364,"driver":{"public_dns":"ec2-52-36-219-62.us-west-2.compute.amazonaws.com","ui_uri":"http://10.172.255.139:41997","memory_mb":10347,"node_id":"318b0e6b79a44b1ba397bd8517f48bde","node_aws_attributes":{"is_spot":false},"instance_id":"i-0aab5c13305b8bdc9","start_timestamp":1606252820927,"host_private_ip":"10.172.247.189","private_ip":"10.172.255.139"},"driver_uri":"https://10.172.255.139:6060","driver_ui_uri":"http://10.172.255.139:41997","spark_context_id":7411043853808608843,"driver_healthy":true,"jdbc_port":10000,"cluster_name":"final '
 'proj","spark_version":"7.4.x-cpu-ml-scala2.12","spark_conf":{"spark.databricks.delta.preview.enabled":"true"},"cluster_creator":"WEBAPP","aws_attributes":{"zone_id":"us-west-2c","first_on_demand":0,"availability":"ON_DEMAND","spot_bid_price_percent":100,"ebs_volume_count":0},"node_type_id":"dev-tier-node","driver_node_type_id":"dev-tier-node","autotermination_minutes":120,"enable_elastic_disk":false,"disk_spec":{"disk_count":0},"cluster_source":"UI","enable_jdbc_auto_start":true,"enable_jobs_autostart":true,"enable_local_disk_encryption":false,"package_label":"release__7.4.x-snapshot-cpu-ml-scala2.12__databricks-universe__head__ff14780__5233a8c__jenkins__2703f5a__format-2","state":"RUNNING","state_message":"","start_time":1606252820551,"terminated_time":0,"last_state_loss_time":0,"last_command_completion_time":1606252890626,"last_activity_time":1606252890626,"creator_org_id":7581726143880285,"num_workers":0,"cluster_memory_mb":15616,"cluster_cores":2.0,"creator_user_name":"prodbyskimmer@gmail.com","acl_path":"/clusters/1124-212020-lark668","init_scripts_safe_mode":false}',
 'mlflow.databricks.notebookID': '929351302293044',
 'mlflow.databricks.notebookPath': '/Users/prodbyskimmer@gmail.com/Final '
 'Research Project/05 - Random Forest',
 'mlflow.databricks.notebookRevisionID': '1606257899128',
 'mlflow.databricks.webappURL': 'https://community.cloud.databricks.com',
 'mlflow.parentRunId': '8aed3e37a8de49cdbfcb1ac09429ad40',
 'mlflow.rootRunId': '8aed3e37a8de49cdbfcb1ac09429ad40',
 'mlflow.source.name': '/Users/prodbyskimmer@gmail.com/Final Research '
 'Project/05 - Random Forest',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'prodbyskimmer@gmail.com'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302293044/6c4535259c8a4a97a2c438174047f9c8/artifacts', end_time=1606257899000, experiment_id='929351302293044', lifecycle_stage='active', run_id='6c4535259c8a4a97a2c438174047f9c8', run_uuid='6c4535259c8a4a97a2c438174047f9c8', start_time=1606257744476, status='FINISHED', user_id=''>>

rmse
0.5858820704844101


## White Wine

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    # hyperparameters to tune 
    maxDepth = params['maxDepth']
    maxBins = params['maxBins']
    numTrees = params['numTrees']
    
    # model 
    vecAssemblerRF = VectorAssembler(inputCols = [x for x in whiteTrainDF.columns if x != 'quality'], outputCol = 'features')
    rf = RandomForestRegressor(labelCol = 'quality', maxBins = maxBins, maxDepth = maxDepth, numTrees = numTrees, seed = 42)
    
    # pipeline 
    pipeline = Pipeline(stages = [vecAssemblerRF, rf])
    pipelineModel = pipeline.fit(whiteTrainDF)
    
    # evaluate predictions 
    whitePredDF = pipelineModel.transform(whiteTestDF)
    regressionEvaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
    rmse = regressionEvaluator.evaluate(whitePredDF)
    
    # log parameters
    mlflow.log_param('maxDepth', maxDepth)
    mlflow.log_param('maxBins', maxBins)
    mlflow.log_param('numTrees', numTrees)
    mlflow.log_param('color', 'white')
    mlflow.log_param('features', 'all')
    mlflow.log_param('tuning', 'hyperopt')
    mlflow.log_metric('rmse', rmse)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'WHITE-RF-All-Features-Hyperopt') as run:
  num_evals = 10
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = search_space,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  # log param and metric for best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])

0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [07:47<1:10:06, 467.35s/trial, best loss: 0.6510414239451864] 20%|██ | 2/10 [19:50<1:12:33, 544.20s/trial, best loss: 0.6426055598455074] 30%|███ | 3/10 [28:14<1:02:04, 532.11s/trial, best loss: 0.6426055598455074] 40%|████ | 4/10 [30:50<41:55, 419.29s/trial, best loss: 0.6426055598455074]  50%|█████ | 5/10 [31:25<25:19, 303.96s/trial, best loss: 0.6426055598455074] 60%|██████ | 6/10 [31:31<14:18, 214.67s/trial, best loss: 0.6426055598455074] 70%|███████ | 7/10 [37:45<13:06, 262.29s/trial, best loss: 0.6426055598455074] 80%|████████ | 8/10 [47:53<12:11, 365.99s/trial, best loss: 0.6426055598455074] 90%|█████████ | 9/10 [53:45<06:01, 361.78s/trial, best loss: 0.6426055598455074]100%|██████████| 10/10 [1:12:31<00:00, 591.16s/trial, best loss: 0.63561053499133]100%|██████████| 10/10 [1:12:31<00:00, 435.17s/trial, best loss: 0.63561053499133]

In [0]:
whiteResult = pd.DataFrame(best_hyperparam, index = [0])
display(whiteResult)

maxBins,maxDepth,numTrees
21,29,94


In [0]:
bestMaxBins = best_hyperparam['maxBins']
bestMaxDepth = best_hyperparam['maxDepth']
bestNumTrees = best_hyperparam['numTrees']

In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['tuning'] == 'hyperopt' and int(run.data.params['maxBins']) == bestMaxBins and int(run.data.params['maxDepth']) == bestMaxDepth and int(run.data.params['numTrees']) == bestNumTrees:
    print(run) 
    whiteMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(whiteMetricsDF)

<Run: data=<RunData: metrics={'rmse': 0.63561053499133}, params={'color': 'white',
 'features': 'all',
 'maxBins': '21',
 'maxDepth': '29',
 'numTrees': '94',
 'tuning': 'hyperopt'}, tags={'mlflow.databricks.cluster.id': '1124-235309-gins728',
 'mlflow.databricks.cluster.info': '{"cluster_id":"1124-235309-gins728","creator_user_id":2440320843892364,"driver":{"public_dns":"ec2-52-36-219-62.us-west-2.compute.amazonaws.com","ui_uri":"http://10.172.233.52:43661","memory_mb":10347,"node_id":"d8a92f58d19d4884a54dfebcc18afb58","node_aws_attributes":{"is_spot":false},"instance_id":"i-0aab5c13305b8bdc9","start_timestamp":1606261989912,"host_private_ip":"10.172.247.189","private_ip":"10.172.233.52"},"driver_uri":"https://10.172.233.52:6060","driver_ui_uri":"http://10.172.233.52:43661","spark_context_id":2197286493742322252,"driver_healthy":true,"jdbc_port":10000,"cluster_name":"final '
 'proj '
 '(clone)","spark_version":"7.4.x-cpu-ml-scala2.12","spark_conf":{"spark.databricks.delta.preview.enabled":"true"},"cluster_creator":"WEBAPP","aws_attributes":{"zone_id":"us-west-2c","first_on_demand":0,"availability":"ON_DEMAND","spot_bid_price_percent":100,"ebs_volume_count":0},"node_type_id":"dev-tier-node","driver_node_type_id":"dev-tier-node","autotermination_minutes":120,"enable_elastic_disk":false,"disk_spec":{"disk_count":0},"cluster_source":"UI","enable_jdbc_auto_start":true,"enable_jobs_autostart":true,"enable_local_disk_encryption":false,"package_label":"release__7.4.x-snapshot-cpu-ml-scala2.12__databricks-universe__head__ff14780__5233a8c__jenkins__2703f5a__format-2","state":"RUNNING","state_message":"","start_time":1606261989550,"terminated_time":0,"last_state_loss_time":0,"last_command_completion_time":1606262008900,"last_activity_time":1606262008900,"creator_org_id":7581726143880285,"num_workers":0,"cluster_memory_mb":15616,"cluster_cores":2.0,"creator_user_name":"prodbyskimmer@gmail.com","acl_path":"/clusters/1124-235309-gins728","init_scripts_safe_mode":false}',
 'mlflow.databricks.notebookID': '929351302293044',
 'mlflow.databricks.notebookPath': '/Users/prodbyskimmer@gmail.com/Final '
 'Research Project/05 - Random Forest',
 'mlflow.databricks.notebookRevisionID': '1606266911257',
 'mlflow.databricks.webappURL': 'https://community.cloud.databricks.com',
 'mlflow.parentRunId': '04cd5c8093f64b0fb9baf646ff5d977a',
 'mlflow.rootRunId': '04cd5c8093f64b0fb9baf646ff5d977a',
 'mlflow.source.name': '/Users/prodbyskimmer@gmail.com/Final Research '
 'Project/05 - Random Forest',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'prodbyskimmer@gmail.com'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302293044/eb3a4d48add44d8a8def8dd02f55355c/artifacts', end_time=1606266911142, experiment_id='929351302293044', lifecycle_stage='active', run_id='eb3a4d48add44d8a8def8dd02f55355c', run_uuid='eb3a4d48add44d8a8def8dd02f55355c', start_time=1606265784866, status='FINISHED', user_id=''>>

rmse
0.63561053499133


## with Feature Selection (Test)

## Red Wine

In [0]:
def objective_function(params):
  maxDepth = params['maxDepth']
  maxBins = params['maxBins']
  numTrees = params['numTrees']
  
  # model 
  vecAssembler = VectorAssembler(inputCols = [x for x in redTrainDF.columns if x != 'quality' and x != 'pH' and x != 'free_sulfur_dioxide' and x != 'citric_acid' and x != 'residual_sugar'], outputCol = 'features')
  rf = RandomForestRegressor(labelCol = 'quality', maxBins = maxBins, maxDepth = maxDepth, numTrees = numTrees, seed = 42)
  
  # pipeline 
  pipeline = Pipeline(stages = [vecAssembler, rf])
  pipelineModel = pipeline.fit(redTrainDF)
  
  # evaluate predictions 
  redPredDF = pipelineModel.transform(redTestDF) 
  regressionEvaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
  rmse = regressionEvaluator.setMetricName('rmse').evaluate(redPredDF)
  
  return {'loss': rmse, 'status': STATUS_OK}

num_evals = 30 
trials = Trials() 
best_hyperparam = fmin(fn = objective_function, 
                      space = search_space, 
                      algo = tpe.suggest,  
                      max_evals = num_evals,
                      trials = trials)
print(trials.best_trial['result']['loss'])

0%| | 0/30 [00:00<?, ?trial/s, best loss=?] 3%|▎ | 1/30 [00:15<07:16, 15.06s/trial, best loss: 0.6260289017220526] 7%|▋ | 2/30 [03:48<34:47, 74.54s/trial, best loss: 0.6036298223831581] 10%|█ | 3/30 [04:46<31:17, 69.55s/trial, best loss: 0.6036298223831581] 13%|█▎ | 4/30 [04:55<22:16, 51.42s/trial, best loss: 0.6036298223831581] 17%|█▋ | 5/30 [05:55<22:27, 53.89s/trial, best loss: 0.6036298223831581] 20%|██ | 6/30 [06:20<18:10, 45.46s/trial, best loss: 0.6036298223831581] 23%|██▎ | 7/30 [08:20<25:58, 67.76s/trial, best loss: 0.6036298223831581] 27%|██▋ | 8/30 [12:00<41:33, 113.34s/trial, best loss: 0.6006803276436066] 30%|███ | 9/30 [12:04<28:13, 80.64s/trial, best loss: 0.6006803276436066]  33%|███▎ | 10/30 [12:08<19:12, 57.61s/trial, best loss: 0.6006803276436066] 37%|███▋ | 11/30 [16:13<35:59, 113.68s/trial, best loss: 0.6006803276436066] 40%|████ | 12/30 [16:17<24:18, 81.00s/trial, best loss: 0.6006803276436066]  43%|████▎ | 13/30 [16:22<16:27, 58.09s/trial, best loss: 0.6006803276436066] 47%|████▋ | 14/30 [18:47<22:26, 84.17s/trial, best loss: 0.5993170779016336] 50%|█████ | 15/30 [18:57<15:30, 62.06s/trial, best loss: 0.5993170779016336] 53%|█████▎ | 16/30 [21:44<21:45, 93.27s/trial, best loss: 0.5993170779016336] 57%|█████▋ | 17/30 [21:47<14:23, 66.42s/trial, best loss: 0.5993170779016336] 60%|██████ | 18/30 [24:50<20:15, 101.25s/trial, best loss: 0.5993170779016336] 63%|██████▎ | 19/30 [26:34<18:42, 102.04s/trial, best loss: 0.5993170779016336] 67%|██████▋ | 20/30 [26:51<12:45, 76.60s/trial, best loss: 0.5993170779016336]  70%|███████ | 21/30 [29:17<14:37, 97.48s/trial, best loss: 0.5993170779016336] 73%|███████▎ | 22/30 [31:22<14:04, 105.62s/trial, best loss: 0.5993170779016336] 77%|███████▋ | 23/30 [32:52<11:46, 100.96s/trial, best loss: 0.5949935419529006] 80%|████████ | 24/30 [33:30<08:12, 82.02s/trial, best loss: 0.5949935419529006]  83%|████████▎ | 25/30 [36:01<08:34, 102.89s/trial, best loss: 0.5949935419529006] 87%|████████▋ | 26/30 [36:15<05:05, 76.26s/trial, best loss: 0.5949935419529006]  90%|█████████ | 27/30 [37:17<03:35, 71.97s/trial, best loss: 0.5949935419529006] 93%|█████████▎| 28/30 [37:25<01:45, 52.55s/trial, best loss: 0.5949935419529006] 97%|█████████▋| 29/30 [41:41<01:53, 113.76s/trial, best loss: 0.5949935419529006]100%|██████████| 30/30 [45:37<00:00, 150.24s/trial, best loss: 0.5949935419529006]100%|██████████| 30/30 [45:37<00:00, 91.23s/trial, best loss: 0.5949935419529006] 
0.5949935419529006

In [0]:
print(best_hyperparam)

{'maxBins': 32, 'maxDepth': 14, 'numTrees': 59}

## White Wine

In [0]:
def objective_function(params):
  maxDepth = params['maxDepth']
  maxBins = params['maxBins']
  numTrees = params['numTrees']
  
  # model 
  vecAssembler = VectorAssembler(inputCols = [x for x in whiteTrainDF.columns if x != 'quality' and x != 'citric_acid' and x != 'residual_sugar' and x != 'fixed_acidity' and x != 'sulphates'], outputCol = 'features')
  rf = RandomForestRegressor(labelCol = 'quality', maxBins = maxBins, maxDepth = maxDepth, numTrees = numTrees, seed = 42)
  
  # pipeline 
  pipeline = Pipeline(stages = [vecAssembler, rf])
  pipelineModel = pipeline.fit(whiteTrainDF)
  
  # evaluate predictions  
  whitePredDF = pipelineModel.transform(whiteTestDF)  
  regressionEvaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality') 
  rmse = regressionEvaluator.setMetricName('rmse').evaluate(whitePredDF) 
  
  return {'loss': rmse, 'status': STATUS_OK}

num_evals = 10 
trials = Trials()
best_hyperparam = fmin(fn = objective_function, 
                      space = search_space, 
                      algo = tpe.suggest, 
                      max_evals = num_evals,
                      trials = trials)
print(trials.best_trial['result']['loss'])

0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [06:25<57:49, 385.47s/trial, best loss: 0.6522779849730048] 20%|██ | 2/10 [17:47<1:03:16, 474.52s/trial, best loss: 0.6522779849730048] 30%|███ | 3/10 [35:42<1:16:22, 654.65s/trial, best loss: 0.6506033749697562] 40%|████ | 4/10 [42:56<58:50, 588.34s/trial, best loss: 0.6506033749697562]  50%|█████ | 5/10 [46:27<39:35, 475.09s/trial, best loss: 0.6506033749697562] 60%|██████ | 6/10 [52:02<28:52, 433.16s/trial, best loss: 0.6506033749697562] 70%|███████ | 7/10 [53:04<16:05, 321.69s/trial, best loss: 0.6506033749697562] 80%|████████ | 8/10 [54:18<08:14, 247.47s/trial, best loss: 0.6506033749697562] 90%|█████████ | 9/10 [54:28<02:56, 176.39s/trial, best loss: 0.6506033749697562]100%|██████████| 10/10 [57:16<00:00, 173.83s/trial, best loss: 0.6506033749697562]100%|██████████| 10/10 [57:16<00:00, 343.68s/trial, best loss: 0.6506033749697562]
0.6506033749697562

In [0]:
print(best_hyperparam)

{'maxBins': 33, 'maxDepth': 28, 'numTrees': 93}